In [29]:
import os
import glob
import pandas as pd
import numpy as np
import random
import torch
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

# import bioacoustics_model_zoo as bmz

In [4]:
# Stratified Verison

# Use maximum CPUs for your device
num_workers = os.cpu_count() * 3 // 4  # Use 75% of cores
print(f"CPU CORES: {num_workers}")

model = bmz.load('BirdNET')  # or bmz.load('Perch')

fold_num = 5  # How many folds
skf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=8)

ROC_AUC_scores = []

for fold_idx, (train_idx, test_idx) in enumerate(skf.split(labels, labels)):  # StratifiedKFold needs labels for stratification
    labels_train = labels.iloc[train_idx]
    labels_val = labels.iloc[test_idx]

    emb_train = model.embed(labels_train, return_dfs=False, batch_size=4, num_workers=num_workers)  # Made Batch Size Smaller
    emb_val = model.embed(labels_val, return_dfs=False, batch_size=4, num_workers=num_workers)
    
    classes = ['A']
    birdnet.change_classes(classes)
    
    # Fit the classification head with embeddings and labels
    birdnet.network.fit(emb_train, labels_train.values, emb_val, labels_val.values)

    # Make predictions by passing the embeddings through the classifier
    preds = birdnet.network(torch.tensor(emb_val)).detach()
    
    # Calculate the area under the ROC score
    curr_score = roc_auc_score(labels_val.values, preds, average=None) 
    ROC_AUC_scores.append(curr_score)
    
    preds = preds.detach().numpy()
    plt.hist(preds[labels_val == True], bins=20, alpha=0.5, label='positives')
    plt.hist(preds[labels_val == False], bins=20, alpha=0.5, label='negatives')
    plt.title(f"Fold {fold_idx}")
    plt.legend()
    
    print('---------------------------')
    print(f"Current Fold: {fold_idx}")
    print(f"ROC AUC Score: {curr_score}")


CPU CORES: 9


NameError: name 'bmz' is not defined

In [21]:
# Haven't Ran yet need packages
# need Module Open SoundScape

# Use maximum cpu's for your device
num_workers = os.cpu_count() * 3 // 4 # Use 75% of cores
print(f"CPU CORES: {num_workers}")



model = bmz.load('BirdNET') #or bmz.load('Perch')

fold_num = 5 # How many folds
kf = KFold(n_splits=fold_num, shuffle=True, random_state=8)

ROC_AUC_scores = []

for fold_idx, (train_idx, test_idx) in enumerate(kf.split(labels)):
    
    labels_train = labels.iloc[train_idx] 
    labels_val = labels.iloc[test_idx]

    emb_train = model.embed(labels_train, return_dfs=False, batch_size=4, num_workers=num_workers) # Made Batch Size Smaller, 128 processes everything at once
    emb_val = model.embed(labels_val, return_dfs=False, batch_size=4, num_workers=num_workers)
    
    classes = ['A']
    birdnet.change_classes(classes) 
    
    # fit the classification head with embeddings and labels
    birdnet.network.fit(emb_train, labels_train.values, emb_val, labels_val.values)

    # make predictions by passing the embeddings through the classifier
    preds = birdnet.network(torch.tensor(emb_val)).detach()
    
    # calculate the area under the ROC score
    # Scikit-Learn uses Mann-Whitney U test for ROC-AUC score
    curr_score = roc_auc_score(labels_val.values, preds, average=None) 
    ROC_AUC_score.append(curr_score)
    
    

    preds = preds.detach().numpy()
    plt.hist(preds[labels_val==True], bins=20,alpha=0.5, label='positives')
    plt.hist(preds[labels_val==False], bins=20,alpha=0.5, label='negatives')
    plt.title(f"Fold {fold_idx}")
    plt.legend()
    
    print('---------------------------')
    print(f"Current Fold: {fold_idx}")
    print(f"ROC AUC Score: {curr_score}")

CPU CORES: 9


NameError: name 'bmz' is not defined